# EDA de **Televisión por Suscripción: Accesos Totales (con ID)**

Cuaderno listo para **Visual Studio/VS Code** (Python) con explicaciones en Markdown entre pasos y guardado del CSV 'limpio'.

**Fuente de datos (ruta Windows):**

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\TV\\Televisión Accesos Totales_con_ID.csv`


## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
Se usa `encoding='utf-8-sig'` para acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\TV\\Televisión Accesos Totales_con_ID.csv'

# Lectura
df = pd.read_csv(ruta, encoding='utf-8-sig')
# Normalizar nombres y renombrar
df.columns = [c.strip().replace('﻿','') for c in df.columns]
col_map = {
    'Registro_ID': 'Registro_ID',
    'Año': 'Año',
    'Trimestre': 'Trimestre',
    'TV por Suscripción (VFR)': 'TV_VFR',
    'TV Satelital': 'TV_Satelital',
    'TiempoTrim_ID': 'TiempoTrim_ID'
}
for c in list(df.columns):
    if 'VFR' in c: col_map[c] = 'TV_VFR'
    if 'Satelital' in c: col_map[c] = 'TV_Satelital'
df = df.rename(columns=col_map)

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial
- Tipos de datos y memoria (`df.info()`).
- Nulos y duplicados.
- Rangos básicos por VFR/Satelital.

In [ ]:
df.info()

print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

for c in ['TV_VFR','TV_Satelital']:
    if c in df.columns:
        print(c, 'min=', df[c].min(), 'max=', df[c].max(), 'mean=', float(df[c].mean()))

## 4. Validaciones y consistencia
- Conversión a **numérico** de columnas clave.
- Chequeo de **negativos**.
- Construcción de `Periodo` (YYYY-Qn) y orden temporal.

In [ ]:
num_cols = ['Registro_ID','Año','Trimestre','TV_VFR','TV_Satelital','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())

df = df.drop_duplicates().sort_values(['Año','Trimestre']).reset_index(drop=True)
df['Periodo'] = pd.PeriodIndex.from_fields(year=df['Año'], quarter=df['Trimestre']).astype(str)

## 5. Variables derivadas
- `Total` = VFR + Satelital.
- `share_vfr_%` y `share_sat_%`.
- Variaciones `QoQ_%`, `YoY_%` y `MM4` para Total y componentes.

In [ ]:
df['Total'] = df['TV_VFR'] + df['TV_Satelital']
total_nonzero = df['Total'].replace(0, pd.NA)
df['share_vfr_%'] = (df['TV_VFR'] / total_nonzero) * 100
df['share_sat_%'] = (df['TV_Satelital'] / total_nonzero) * 100

for base, col in [('VFR','TV_VFR'),('Sat','TV_Satelital'),('Total','Total')]:
    df[f'{base}_QoQ_%'] = df[col].pct_change() * 100
    df[f'{base}_YoY_%'] = df[col].pct_change(4) * 100
    df[f'{base}_MM4'] = df[col].rolling(4).mean()

display(df[['Año','Trimestre','TV_VFR','TV_Satelital','Total','share_vfr_%','share_sat_%','Total_QoQ_%','Total_YoY_%','Total_MM4']].tail(8))

## 6. Visualizaciones
- **Serie temporal** de Total y su **media móvil (4)**.
- **Stacked area** VFR vs Satelital.
- **Participación (%)** de VFR vs Satelital (últimos 12 trimestres).
- **YoY** y **QoQ** de Total.

In [ ]:
import matplotlib.ticker as mtick

# 6.1 Total + MM4
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Total', marker='o', label='Total', color='#2E86C1')
sns.lineplot(data=df.dropna(subset=['Total_MM4']), x='Periodo', y='Total_MM4', marker='o', label='MM4 Total', color='#7D3C98')
plt.xticks(rotation=45, ha='right'); plt.title('Accesos totales de TV por suscripción'); plt.xlabel('Periodo'); plt.ylabel('Accesos'); plt.legend(); plt.tight_layout(); plt.show()

# 6.2 Stacked area VFR vs Sat
plt.figure()
plt.stackplot(df['Periodo'], df['TV_VFR'], df['TV_Satelital'], labels=['VFR','Satelital'], colors=['#1ABC9C','#E67E22'])
plt.xticks(rotation=45, ha='right'); plt.title('Composición de accesos (VFR vs Satelital)'); plt.xlabel('Periodo'); plt.ylabel('Accesos'); plt.legend(loc='upper left'); plt.tight_layout(); plt.show()

# 6.3 Participación (%) últimos 12
tail = df.tail(12)
plt.figure()
sns.lineplot(data=tail, x='Periodo', y='share_vfr_%', marker='o', label='VFR %', color='#1ABC9C')
sns.lineplot(data=tail, x='Periodo', y='share_sat_%', marker='o', label='Satelital %', color='#E67E22')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter()); plt.xticks(rotation=45, ha='right'); plt.title('Participación (%) de VFR vs Satelital (últimos 12 trimestres)'); plt.xlabel('Periodo'); plt.ylabel('% del total'); plt.legend(); plt.tight_layout(); plt.show()

# 6.4 YoY y QoQ del Total
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Total_YoY_%', marker='o', label='YoY %', color='#34495E')
sns.lineplot(data=df, x='Periodo', y='Total_QoQ_%', marker='o', label='QoQ %', color='#D35400')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter()); plt.xticks(rotation=45, ha='right'); plt.title('Variación del Total (YoY y QoQ)'); plt.xlabel('Periodo'); plt.ylabel('%'); plt.legend(); plt.tight_layout(); plt.show()

## 7. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)